### Word Embeddings

En esencia, las representaciones de palabras (word embeddings) son representaciones numéricas de datos del mundo real como texto, imágenes, voz o video. Se expresan como vectores de baja dimensionalidad, donde las distancias geométricas de dos vectores en el espacio vectorial son una proyección de las relaciones entre los dos objetos del mundo real que los vectores representan. En otras palabras, te ayudan a proporcionar una representación compacta de datos de diferentes tipos, mientras que simultáneamente te permiten comparar dos objetos de datos diferentes y decir cuán similares o diferentes son en una escala numérica.

Estas representaciones de baja dimensionalidad de datos del mundo real ayudan significativamente a un procesamiento y almacenamiento de datos a gran escala de manera eficiente, actuando como un medio de compresión con pérdida de los datos originales mientras se conservan sus propiedades importantes.

Los "word embeddings" se construyen con diferentes técnica y con modelos de aprendizaje automático, generalmente se pueden usar los word embeddings creados por ChatGPT (OpenAI), Google AI etc.





In [1]:
!pip3 install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 2.1/203.0 MB 9.8 MB/s eta 0:00:21
    --------------------------------------- 3.9/203.0 MB 9.0 MB/s eta 0:00:23
   - -------------------------------------- 6.3/203.0 MB 9.9 MB/s eta 0:00:20
   - -------------------------------------- 8.4/203.0 MB 9.8 MB/s eta 0:00:20
   -- ------------------------------------- 10.7/203.0 MB 10.0 MB/s eta 0:00:20
   -- ------------------------------------- 13.4/203.0 MB 10.4 MB

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
#from dotenv import load_dotenv
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings
from InstructorEmbedding import INSTRUCTOR


c:\Users\L03564299\AppData\Local\miniconda3\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


### Text embeddings 



In [6]:
#embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')  #Problem with Python 12
#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [7]:
input_text = "happy"
vector = embeddings.embed_query(input_text)
print(vector)

[-0.7404394149780273, -1.803025245666504, 4.060121059417725, -0.8889495730400085, 0.2498321235179901, 0.962736964225769, 1.561875581741333, -0.7627670764923096, -0.6247386932373047, -1.2378244400024414, 0.04657385125756264, 0.5642500519752502, 3.319350242614746, 1.2903879880905151, -1.0973732471466064, -1.1954660415649414, 1.4598197937011719, -0.05195207893848419, 0.6194881796836853, 1.0011619329452515, -1.7244963645935059, -1.2395309209823608, 1.5592174530029297, 0.01529963780194521, 1.4213370084762573, 0.41600483655929565, 0.8442690968513489, -2.175764322280884, 0.7842884063720703, 0.54567950963974, 1.3880348205566406, 0.2598465383052826, -0.9658294320106506, 2.203990936279297, 0.4654935598373413, -0.36726143956184387, 0.1535971462726593, 0.855857253074646, -0.6525095105171204, 0.36654409766197205, 1.054890513420105, -1.6926548480987549, 3.3145482540130615, -0.6008889675140381, -0.9950042963027954, 0.9266526103019714, -0.7899807095527649, 0.26270362734794617, 0.89438796043396, -1.983

In [9]:
input_text = "Today I am happy"
vector = embeddings.embed_query(input_text)
print(vector)

[-0.3830258548259735, -0.31182101368904114, 4.646028518676758, -0.7912827730178833, -0.3810964524745941, -0.48601022362709045, 1.4306904077529907, 0.04653507471084595, 0.24543750286102295, -0.675326406955719, -0.30564048886299133, -0.37109407782554626, 1.7531663179397583, 1.9200289249420166, 0.9448559284210205, 1.6564337015151978, 1.1297881603240967, -1.324067234992981, 0.43169426918029785, 0.20937509834766388, -1.8435941934585571, -0.8672895431518555, 2.8181471824645996, -0.5931394696235657, -0.43622785806655884, -0.595561146736145, 3.033567190170288, -2.567687749862671, 2.6689882278442383, 0.5217175483703613, 1.031145453453064, 0.4931899905204773, 0.034657981246709824, 0.3744562268257141, 0.6116977334022522, -0.008993696421384811, -1.087975263595581, 2.436589002609253, -0.9069212079048157, 1.7450343370437622, -0.8620107769966125, -0.48102042078971863, 0.5372318625450134, 1.5532275438308716, -1.7458878755569458, 1.8065487146377563, -0.194774329662323, 1.1312943696975708, 0.87911707162

### Vectorstore

Los embeddings se guardan en databases especiales (aquí se usa FAISS) que permiten almacenar y extraer de manera eficiente la información de los embeddings, comparando  el significado semántico de los queries con el significado semántico de las palabras almacenadas en los embeddings.

### Carga y lee el documento pdf

In [10]:
pdf_path="PeterPan.pdf"

In [11]:
text =''
with open(pdf_path,'rb') as file:
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()


In [12]:
print(text)

Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba 
formada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin 
olvidarnos de Nana, por supuesto, el perro niñera.  
 
Vivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la 
noche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. 
Cuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. 
Pero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. 
Entonces apareció Nana, que cerró la ventana para evitar que saliera y acabó 
atrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la 
casa de los Darling.  
 
Una noche el señor y la señora Darling salieron a cenar a casa de los vecinos del 
número 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en 
quedarse todos dormidos.  
Pero cuando la casa estaba en silencio, entró una diminuta

### Divide el texto 

In [13]:
#text_splitter=SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

In [14]:
docs = text_splitter.split_text(text)

In [15]:
type(docs)

list

In [16]:
len(docs)

12

In [17]:
docs[0]

'Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba \nformada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin \nolvidarnos de Nana, por supuesto, el perro niñera.  \n \nVivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la \nnoche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. \nCuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. \nPero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. \nEntonces apareció Nana, que cerró la ventana para evitar que saliera y acabó \natrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la \ncasa de los Darling.  \n \nUna noche el señor y la señora Darling salieron a cenar a casa de los vecinos del \nnúmero 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en \nquedarse todos dormidos.'

In [22]:
docs[3]

'encima y enseguida los tres niños comenzaron a elevarse por el aire. A t odos les \npareció muy divertido y comenzaron a dar vueltas y más vueltas por la casa. Armaron \ntal revuelo que acabaron despertando a Nana.  \n \nPeter la oyó venir así que pudieron volver a sus camas rápidamente como si no \nhubiese pasado nada. Así, cuando la niñ era entró en la habitación creyó que los tres \ndormían plácidamente.  \n \nPero Nana estaba intranquila y estaba casi segura de que algo raro estaba ocurriendo \nen el cuarto de los niños, de modo que corrió a avisar a los señores Darling. Pero \ncuando volvieron, l os niños ya no estaban. Los tres habían partido rumbo a Nunca \nJamás nerviosos e ilusionados por vivir aquella fantástica aventura.  \n \nVolaron durante días atravesando océanos pero al final llegaron al país de Nunca \nJamas.  \n \nAl primero que vieron desde el aire  fue al temible capitán Garfio, el peor enemigo de \nPeter Pan. En una lucha hacía tiempo Peter había logrado arrebatar

### Crea el vectorstore

In [19]:
vectorstore = FAISS.from_texts(docs, embeddings)

### Guardar el vectorstore


In [20]:
vectorstore.save_local("faiss_index")

### Cargar el vectorstore

In [21]:
vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)